# CNN Architecture (Part 3.4)
### This notebook prepares, trains and tests the Convolutional Neural Network (CNN) model architecture for sentiment classification

In [ ]:
! pip install datasets
! pip install nltk
! pip install gensim
! pip install huggingface-hub
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("popular")
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import gensim.downloader as api
import numpy as np
from string import punctuation
from datasets import load_dataset
import string
from nltk.corpus import stopwords
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from nltk.tokenize import word_tokenize
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
import seaborn as sns
import math
import random
from itertools import product
import gc
import copy
import torch.nn.functional as F


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    | 

In [ ]:
from datasets import load_dataset

# Load the Rotten Tomatoes dataset with train, validation, and test splits
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']           # Training set
validation_dataset = dataset['validation'] # Validation set
test_dataset = dataset['test']             # Test set

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
# load a pre-trained Word2Vec model (e.g. "word2vec-google-news-300") from gensim's api
w2vmodel = api.load("fasttext-wiki-news-subwords-300")

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
# remove punctuation and stopwords
import string
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def tokenize_and_remove_punctuation_and_stopwords(text):
    tokens = word_tokenize(text)  # Tokenize the text
    tokens_no_punctuation_or_stopwords = [token for token in tokens if token not in string.punctuation and token not in stop_words]  # Remove punctuation
    return tokens_no_punctuation_or_stopwords

new_tokenized_texts = [tokenize_and_remove_punctuation_and_stopwords(text) for text in train_dataset['text']]

In [ ]:
# get all possible subwords of a word
def get_subwords(word):
    return [word[i:i+n] for n in range(1, len(word) + 1) for i in range(len(word) - n + 1)]

# Training and Evaluation Functions with Early Stopping and Visualization

This code includes the following main components:
1. **EarlyStopper**: A class for implementing early stopping, which helps prevent overfitting by stopping training when validation accuracy stops improving.
2. **train_model**: A function for training a model with optional OOV (out-of-vocabulary) handling, early stopping, and tracking training/validation loss and accuracy.
3. **evaluate_model**: A function for evaluating the model on a test set, displaying a confusion matrix, and calculating accuracy, precision, and F1 score.
4. **plot_loss**: A function for plotting training/validation loss and accuracy curves over epochs for visual analysis.

In [ ]:
# Class for early stopping to prevent overfitting when validation accuracy does not improve
class EarlyStopper:
    def __init__(self, patience=5, verbose=True):
        self.patience = patience  # Number of epochs to wait for improvement
        self.counter = 0  # Tracks consecutive epochs with no improvement
        self.best_acc = 0  # Best validation accuracy observed so far
        self.verbose = verbose  # If True, print early stopping messages

    def early_stop(self, val_acc, model):
        if val_acc > self.best_acc:
            self.best_acc = val_acc  # Update best accuracy
            self.counter = 0  # Reset counter if validation accuracy improves
        else:
            self.counter += 1  # Increment counter if no improvement

        if self.counter >= self.patience:
            if self.verbose:
                print("Early stopping...")  # Print message if stopping
            return True
        return False

# Function to train the model with early stopping and optional OOV handling
def train_model(model, epochs, criterion, optimizer, train_loader, validation_loader, handle_oov=False, earlystopper=EarlyStopper(patience=5, verbose=True)):
    train_loss_history = []  # Track training loss per epoch
    train_accuracy_history = []  # Track training accuracy per epoch
    val_loss_history = []  # Track validation loss per epoch
    val_accuracy_history = []  # Track validation accuracy per epoch
    best_model = model.state_dict()  # Store best model based on validation accuracy
    best_val_accuracy = 0.0  # Initialize best validation accuracy

    for epoch in range(epochs):
        model.train()  # Set model to training mode
        total_loss = 0
        correct = 0
        total = 0

        # Training loop
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
            if isinstance(labels, list):
                labels = torch.tensor(labels)
            if handle_oov:
                labels = labels.to(device)
            else:
                inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Reset gradients
            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs, 1)  # Get predicted labels
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Count correct predictions
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model parameters

            total_loss += loss.item()

        train_loss = total_loss / len(train_loader)  # Compute average training loss
        train_accuracy = correct / total  # Compute training accuracy
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)

        # Validation loop
        model.eval()  # Set model to evaluation mode
        total_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in validation_loader:
                if isinstance(labels, list):
                    labels = torch.tensor(labels)
                if handle_oov:
                    labels = labels.to(device)
                else:
                    inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loss = criterion(outputs, labels)
                total_loss += loss.item()

        val_loss = total_loss / len(validation_loader)  # Compute average validation loss
        val_accuracy = correct / total  # Compute validation accuracy
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")
        print("-" * 60)

        # Early stopping based on validation accuracy
        if earlystopper.early_stop(val_accuracy, model):
            break

        # Save the best model
        if val_accuracy >= best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = copy.deepcopy(model)

    model = copy.deepcopy(best_model)  # Restore best model

    return train_loss_history, train_accuracy_history, val_loss_history, val_accuracy_history, model

# Function to evaluate the model on the test set with optional OOV handling
def evaluate_model(model, test_loader, handle_oov=False):
    model.eval()  # Set model to evaluation mode
    correct, total = 0, 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            if isinstance(labels, list):
                labels = torch.tensor(labels)
            if handle_oov:
                labels = labels.to(device)
            else:
                inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = correct / total  # Overall accuracy
    print(f'Test Accuracy: {accuracy:.2f}')

    # Plot confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Predicted Negative', 'Predicted Positive'],
                yticklabels=['Actual Negative', 'Actual Positive'])
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    # Calculate and print additional metrics
    TN, FP, FN, TP = cm.ravel()
    precision = precision_score(all_labels, all_predictions, average='binary')
    f1 = f1_score(all_labels, all_predictions, average='binary')

    print(f'True Positives (TP): {TP}')
    print(f'True Negatives (TN): {TN}')
    print(f'False Positives (FP): {FP}')
    print(f'False Negatives (FN): {FN}')
    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'F1 Score: {f1:.2f}')

    return accuracy

# Function to plot training and validation loss and accuracy over epochs
def plot_loss(train_loss, train_accuracy, val_loss, val_accuracy_history, title='Training Curves'):
    epochs = np.arange(1, len(train_loss) + 1)  # List of epoch numbers

    plt.figure(figsize=(12, 6))

    # Plot training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label='Training Loss', color='blue', linewidth=2)
    plt.plot(epochs, val_loss, label='Validation Loss', color='orange', linewidth=2)
    plt.title(f"{title} - Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.xticks(ticks=np.arange(1, len(train_loss) + 1, step=max(1, len(epochs) // 10)))
    plt.legend()
    plt.grid(True)

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy, label='Training Accuracy', color='blue', linewidth=2)
    plt.plot(epochs, val_accuracy_history, label='Validation Accuracy', color='orange', linewidth=2)
    plt.title(f"{title} - Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.xticks(ticks=np.arange(1, len(train_loss) + 1, step=max(1, len(epochs) // 10)))
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch

# Custom Dataset class to handle texts and labels
class SentimentDatasetWithText(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts  # List of preprocessed texts
        self.labels = labels  # Corresponding labels for each text

    def __len__(self):
        return len(self.labels)  # Returns the number of samples in the dataset

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]  # Returns a text-label pair

# Function to create data loaders with OOV handling
def create_data_loaders_with_oov_handling(dataset, batch_size):
    # Preprocess and tokenize texts for train, validation, and test sets
    train_texts = [tokenize_and_remove_punctuation_and_stopwords(text) for text in dataset['train']['text']]
    validation_texts = [tokenize_and_remove_punctuation_and_stopwords(text) for text in dataset['validation']['text']]
    test_texts = [tokenize_and_remove_punctuation_and_stopwords(text) for text in dataset['test']['text']]

    # Create Dataset objects for each split
    train_dataset = SentimentDatasetWithText(train_texts, dataset['train']['label'])
    validation_dataset = SentimentDatasetWithText(validation_texts, dataset['validation']['label'])
    test_dataset = SentimentDatasetWithText(test_texts, dataset['test']['label'])

    # Initialize DataLoaders with a custom collate function for batching
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

    return train_loader, validation_loader, test_loader  # Return DataLoaders for each dataset split

# Custom collate function to handle batching of texts and labels
def collate_fn(batch):
    texts, labels = zip(*batch)  # Separate texts and labels
    return list(texts), torch.tensor(labels)  # Return texts as list and labels as tensor


In [ ]:
# Initialize data loaders for training, validation, and testing sets with a batch size of 64
train_loader, validation_loader, test_loader = create_data_loaders_with_oov_handling(dataset, 64)

# Design of CNN Architecture for Sentence Representation and Sentiment Classification

This CNN model is designed to capture local patterns in text data, which are often essential for sentiment analysis. The architecture leverages convolutional layers with multiple filter sizes to create a rich feature map from the input text, ultimately enabling effective sentence-level sentiment classification. Here’s a breakdown of the model components:

---

## Model Architecture

### 1. Embedding Layer
- **Pre-trained Word Embeddings**:  
  The embedding layer is initialized with pre-trained Word2Vec embeddings, providing the model with semantic information.
- **Trainable or Frozen Embeddings**:  
  The embeddings can either be frozen (non-trainable) or fine-tuned during training, based on the boolean parameter `train_embeddings`. Fine-tuning allows the model to adapt the embeddings to the specific dataset, potentially improving accuracy, while freezing helps retain the original semantic structure and can reduce training time.

### 2. Convolutional Layers with Multiple Filter Sizes
- **Purpose of Convolutions**:  
  The convolutional layers help capture n-grams or local patterns in sentences, such as bi-grams and tri-grams, which can carry key sentiment information.
  
- **Multiple Filter Sizes**:  
  The model uses multiple filter sizes (e.g., 3, 4, and 5) to capture various n-gram features. Smaller filters focus on short phrases, while larger filters detect longer expressions, giving the model flexibility to capture patterns of different lengths.
  
- **Feature Extraction**:  
  Each filter size creates a distinct feature map that emphasizes patterns it detects across the sentence, helping the model in identifying sentiment-bearing structures.

### 3. Fully Connected (FC) Layer with Dropout
- **Combining Features**:  
  After convolution, each feature map undergoes max pooling, retaining the most significant features. The pooled outputs from all filters are then concatenated to form a comprehensive representation of the sentence.
  
- **Dropout Regularization**:  
  A dropout layer is applied before the final fully connected layer to reduce overfitting by discouraging the model from over-relying on specific features, thus enhancing generalization.
  
- **Classification**:  
  The final fully connected layer maps the combined features to sentiment classes, providing the sentiment prediction for the input sentence.

### 4. Handling Out-of-Vocabulary (OOV) Words with Subword Embeddings
- **Subword Encoding**:  
  For words outside the vocabulary, the model decomposes them into subwords, averaging their embeddings to approximate the representation of the OOV word.
  
- **Fallback Mechanism**:  
  If a word lacks recognized subwords, a zero vector is used as a fallback. This approach ensures that unknown words do not introduce noise or disrupt sentence representations.

---

## Operational Mechanism: Sentence Representation and Sentiment Classification

### Forward Pass

1. **Embedding Lookup**:  
   For each word in the sentence, the model retrieves embeddings, using averaged subword embeddings for any OOV words as needed.

2. **Padding and Permutation**:  
   To handle variable sentence lengths, embeddings are padded and then permuted to be compatible with the convolutional layers.

3. **Convolutional Feature Extraction**:  
   Each convolutional layer applies its kernel size across the sentence embeddings, producing feature maps that capture different n-gram patterns linked to sentiment.

4. **Pooling and Concatenation**:  
   After convolution, max pooling retains the most prominent features from each feature map. These pooled outputs are concatenated to form a sentence-level representation.

5. **Dropout and Fully Connected Layer**:  
   The concatenated features pass through a dropout layer and then a fully connected layer, mapping them to the final sentiment class labels.

---

## Design Rationale

This CNN model effectively captures localized patterns (e.g., n-grams) without the sequential dependencies that RNNs require. By using multiple kernel sizes, the model learns a range of phrase-level features associated with sentiment. The architecture is structured to efficiently generate sentence representations through convolution and pooling, and the final fully connected layer performs sentiment classification, making it well-suited for tasks like sentiment analysis.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, embedding_dim, output_size, train_embeddings=False, num_filters=100, filter_sizes=[3, 4, 5], dropout=0.5, word2vec_model=w2vmodel):
        super(CNN, self).__init__()

        # Initialize embeddings using pre-trained Word2Vec vectors, with an option to train or freeze them
        self.embeddings = nn.Embedding.from_pretrained(torch.tensor(word2vec_model.vectors, dtype=torch.float32), freeze=not train_embeddings)

        # Set embedding dimension and define convolutional layers with multiple filter sizes
        self.embedding_dim = embedding_dim
        self.convs = nn.ModuleList([nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=f) for f in filter_sizes])

        # Define a fully connected layer for final classification
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_size)

        # Dropout layer for regularization
        self.dropout = nn.Dropout(dropout)

        # Word-to-index mapping for handling vocabulary
        self.word_to_index = word2vec_model.key_to_index

    def encode_with_subwords(self, word, device):
        # Handle out-of-vocabulary words by averaging embeddings of subwords
        subwords = get_subwords(word)
        subword_indices = [self.word_to_index[subword] for subword in subwords if subword in self.word_to_index]

        # Return average embedding for subwords if found, otherwise return zero vector
        if subword_indices:
            subword_embeddings = self.embeddings(torch.tensor(subword_indices, device=device))
            return subword_embeddings.mean(dim=0)
        return torch.zeros(self.embedding_dim, device=device)

    def forward(self, x):
        device = next(self.parameters()).device
        batch_embeddings = []

        # Encode each word in the sentence; for OOV words, use subword embeddings
        for sentence in x:
            sentence_embeddings = [self.embeddings(torch.tensor(self.word_to_index[word], device=device)) if word is not None and word in self.word_to_index else self.encode_with_subwords(word, device) for word in sentence]
            batch_embeddings.append(torch.stack(sentence_embeddings))

        # Pad sequences and permute dimensions for convolutional layers
        batch_embeddings = torch.nn.utils.rnn.pad_sequence(batch_embeddings, batch_first=True)
        x = batch_embeddings.permute(0, 2, 1)

        # Apply convolution and max pooling for each filter size, then concatenate results
        conv_outs = [F.relu(conv(x)).max(dim=2)[0] for conv in self.convs]
        x = torch.cat(conv_outs, dim=1)

        # Pass through dropout and fully connected layers for final prediction
        return self.fc(self.dropout(x))


In [ ]:
def train_cnn_with_fixed_params(num_filters, filter_sizes, learning_rate, batch_size, num_epochs, dropout, train_embeddings_bool=False):
    # Set embedding dimension and output classes for the CNN model
    embedding_dim = 300
    output_size = 2

    # Print selected hyperparameters
    print(f"Running with fixed parameters: num_filters={num_filters}, filter_sizes={filter_sizes}, "
          f"lr={learning_rate}, batch_size={batch_size}, epochs={num_epochs}, dropout={dropout}")

    # Clear GPU cache if available to ensure efficient memory usage
    if torch.cuda.device_count() >= 1:
        gc.collect()
        torch.cuda.empty_cache()

    # Initialize CNN model with specified parameters
    model = CNN(
        embedding_dim=embedding_dim,
        output_size=output_size,
        word2vec_model=w2vmodel,
        train_embeddings=train_embeddings_bool,
        num_filters=num_filters,
        filter_sizes=filter_sizes
    )

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model and capture loss and accuracy histories
    train_loss_history, train_accuracy_history, val_loss_history, val_accuracy_history, best_model = train_model(
        model, num_epochs, criterion, optimizer, train_loader, validation_loader, handle_oov=True
    )

    # Identify best validation accuracy, training accuracy, and lowest validation loss
    best_val_accuracy = max(val_accuracy_history)
    best_train_accuracy = max(train_accuracy_history)
    best_val_loss = min(val_loss_history)

    # Store results for analysis and evaluation
    results = {
        'train_loss_history': train_loss_history,
        'train_accuracy_history': train_accuracy_history,
        'val_loss_history': val_loss_history,
        'val_accuracy_history': val_accuracy_history,
        'best_val_accuracy': best_val_accuracy,
        'best_train_accuracy': best_train_accuracy,
        'best_val_loss': best_val_loss
    }

    # Return the best model and performance results
    return best_model, results


In [ ]:
import gc
import torch
import torch.optim as optim
from tqdm import tqdm
# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define fixed parameters for the CNN model with trainable embeddings
fixed_params_cnn = {
    'num_filters': 50,  # Number of filters in each convolutional layer
    'filter_sizes': [3, 4, 5],  # Filter sizes for capturing different n-gram features
    'learning_rate': 0.001,  # Learning rate for the optimizer
    'batch_size': 128,  # Batch size for training
    'num_epochs': 100,  # Maximum number of epochs for training
    'dropout': 0.7,  # Dropout rate for regularization
    'train_embeddings_bool': True  # Whether to fine-tune the embeddings during training
}

# Train the CNN model using the specified fixed parameters
cnn_model, results_cnn = train_cnn_with_fixed_params(**fixed_params_cnn)

# Print summary of the fixed parameters and best results achieved during training
print(f"Fixed Parameters for CNN: {fixed_params_cnn}")
print(f"Best Validation Loss for CNN: {results_cnn['best_val_loss']:.4f}")
print(f"Best Validation Accuracy for CNN: {results_cnn['best_val_accuracy']:.4f}")
print(f"Best Training Accuracy for CNN: {results_cnn['best_train_accuracy']:.4f}")

# Plot training and validation loss and accuracy curves over epochs
plot_loss(
    results_cnn['train_loss_history'],
    results_cnn['train_accuracy_history'],
    results_cnn['val_loss_history'],
    results_cnn['val_accuracy_history']
)


Running with fixed parameters: num_filters=50, filter_sizes=[3, 4, 5], lr=0.001, batch_size=128, epochs=100, dropout=0.7


Epoch 1/100: 100%|██████████| 134/134 [05:34<00:00,  2.50s/batch]


Epoch 1/100 | Train Loss: 0.5749 | Train Accuracy: 0.6951 | Val Loss: 0.4790 | Val Accuracy: 0.7767
------------------------------------------------------------


Epoch 2/100: 100%|██████████| 134/134 [05:32<00:00,  2.48s/batch]


Epoch 2/100 | Train Loss: 0.2936 | Train Accuracy: 0.8782 | Val Loss: 0.5252 | Val Accuracy: 0.7542
------------------------------------------------------------


...

Epoch 4/100 | Train Loss: 0.0392 | Train Accuracy: 0.9884 | Val Loss: 0.8422 | Val Accuracy: 0.7552
------------------------------------------------------------


Epoch 5/100: 100%|██████████| 134/134 [05:32<00:00,  2.48s/batch]


Epoch 5/100 | Train Loss: 0.0156 | Train Accuracy: 0.9960 | Val Loss: 0.9568 | Val Accuracy: 0.7552
------------------------------------------------------------


Epoch 6/100: 100%|██████████| 134/134 [05:32<00:00,  2.48s/batch]


Epoch 6/100 | Train Loss: 0.0082 | Train Accuracy: 0.9984 | Val Loss: 1.0330 | Val Accuracy: 0.7589
------------------------------------------------------------
Early stopping...
Fixed Parameters for CNN: {'num_filters': 50, 'filter_sizes': [3, 4, 5], 'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 100, 'dropout': 0.7, 'train_embeddings_bool': True}
Best Validation Loss for CNN: 0.4790
Best Validation Accuracy for CNN: 0.7767
Best Training Accuracy for CNN: 0.9984


### Training Results Analysis (Displayed Above)

The training results for the CNN model with the chosen fixed parameters demonstrate promising accuracy on the training set but highlight overfitting on the validation set. Key observations are as follows:

#### Loss and Accuracy Trends

- **Training Loss and Accuracy**: The training loss drops sharply, reaching close to zero by epoch 6. Correspondingly, the training accuracy approaches 100%, showing that the model has learned the training data very well. This rapid convergence on the training set indicates high model capacity.

- **Validation Loss and Accuracy**: In contrast, the validation loss initially decreases but then begins to increase from around the second epoch onward. The validation accuracy peaks early at 77.67% and fluctuates slightly without significant improvement in later epochs, indicating that the model is overfitting.

#### Justification for Overfitting

Although overfitting is evident, this configuration provided the best validation accuracy across various trials. Alternative setups were tested, including:

1. **Reducing Model Complexity**: Lowering the number of filters or reducing layers mitigated overfitting but significantly decreased validation accuracy, limiting the model’s ability to capture necessary features.
   
2. **Adjusting Dropout and Learning Rates**: Higher dropout rates and alternative learning rates were tried to enhance generalization. However, these adjustments led to underfitting, where both training and validation accuracies declined.
   
3. **Varying Batch Sizes**: Changing the batch size did not improve validation performance meaningfully and often introduced other trade-offs, such as slower convergence.

This configuration, despite some overfitting, achieved the highest validation accuracy among tested setups, making it the optimal choice under current constraints. Early stopping was applied to prevent excessive overfitting, ensuring the model did not train beyond the point where validation accuracy stabilized.

#### Summary

- **Best Validation Accuracy**: 77.67%
- **Best Validation Loss**: 0.4790
- **Best Training Accuracy**: Approached 100%

The model's strong training performance combined with controlled overfitting through early stopping supports this configuration as the best balance between learning capacity and generalization for the task.


In [ ]:
# Evaluate the best model on the test set with OOV (out-of-vocabulary) handling enabled
evaluate_model(cnn_model,test_loader,handle_oov=True)

Test Accuracy: 0.78


True Positives (TP): 424
True Negatives (TN): 403
False Positives (FP): 130
False Negatives (FN): 109
Accuracy: 0.78
Precision: 0.77
F1 Score: 0.78


0.775797373358349

### Test Results Analysis (Displayed Above)

The model’s performance on the test set yielded an overall accuracy of **78%**. Below is an analysis of the confusion matrix and additional metrics:

#### Confusion Matrix Summary

- **True Positives (TP)**: 424  
  Correctly classified positive samples.

- **True Negatives (TN)**: 403  
  Correctly classified negative samples.

- **False Positives (FP)**: 130  
  Misclassified negative samples as positive, suggesting some negative expressions may have been misinterpreted as positive.

- **False Negatives (FN)**: 109  
  Misclassified positive samples as negative, indicating some positive samples with subtle or complex expressions were missed.

#### Performance Metrics

- **Accuracy**: 0.78  
  Indicates that the model correctly classified 78% of the test samples.

- **Precision**: 0.77  
  Of all the samples predicted as positive, 77% were correctly identified. This metric shows that the model maintains a reasonable balance in positive predictions, though some false positives suggest room for improvement.

- **F1 Score**: 0.78  
  A balanced measure that considers both precision and recall, indicating the model’s overall effectiveness in classification.

#### Observations

The results indicate that the model generally performs well, but there is some scope for improvement in handling ambiguous or subtle expressions that may lead to false positives and negatives. These errors suggest potential fine-tuning opportunities, such as additional training on nuanced language or tweaking the threshold for classification to balance precision and recall.

Overall, the model demonstrates reliable performance on the test set, with acceptable accuracy and balanced precision-recall trade-offs.
